# Relatório 8 - Rastreamento de Objetos

Tutoriais utilizados

https://docs.opencv.org/4.5.3/d2/d0a/tutorial_introduction_to_tracker.html

https://learnopencv.com/object-tracking-using-opencv-cpp-python/

https://github.com/spmallick/learnopencv/tree/master/tracking

### Este tutorial foi executado como .py por não conseguir o mesmo processo com o Google Colab, o codigo apresentado nesse notebook foi feito para melhor documentação de cada etapa, e o resultado foi anexado ao final

## Script utilizado para a execução do rastreamento

In [ ]:
# importando as bibliotecas utilizadas no script
import cv2 #opencv
import sys #bibloteca de funções do sistemas, util para navegar,criar e modificar pastas e arquivos

# verificando a versao exata do opencv instalado para identificar a estrtura de inicializacao dos trackers
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

# executa apenas se o script foi executado diretamente
if __name__ == '__main__' :

    # Lista os trackers disponiveis na biblioteca cv2
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'CSRT']
    
    # neste caso utilizaremos apenas o método KCF
    tracker_type = tracker_types[2]

    # diferencia a inicializacao do tracker de acordo com a versao do opencv instalado
    if int(minor_ver) < 3:
        tracker = cv2.Tracker_create(tracker_type)
    else:
        if tracker_type == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        if tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        if tracker_type == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        if tracker_type == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        if tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()

    # realiza a leitura do vídeo que iremos utilizar para realizar a captura
    video = cv2.VideoCapture("sample-mp4.mp4")

    # caso o video não for aberto corretamente o scrip é terminado
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # realiza a leitura do primeiro quadro para selecionar o objeto para ser trackeado, se houver erro na leitura o script é terminado
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()
    
    # inicializa a tupla para a seleção do objeto
    bbox = (287, 23, 86, 320)

    # pede ao usuário que seleciona a area com o objeto a ser rastreado
    bbox = cv2.selectROI(frame, False)

    # inicializa o rastreamento
    ok = tracker.init(frame, bbox)

    # looping de leitura e rastreamento de novos pixels enquanto o video não terminar
    while True:
        # le um novo quadro e termina o script caso nao for possivel le-lo
        ok, frame = video.read()
        if not ok:
            break
        
        # inicializa o timer para calcular o numero de quadros por segundo
        timer = cv2.getTickCount()

        # atualiza o rastreamento no novo quadro
        ok, bbox = tracker.update(frame)

        # calcula o numero de quadros por segundo
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        # desenha a atualizacao da area do objeto rastreado
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # indica a falha na leitura do objeto rastreado
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # mostra o tipo de rastreador que está sendo utilizado
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
    
        # mostra o numero de quadros por segundo
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);


        # atualiza a exibicao com o novo quadro, com o video, o objeto rastreado, o tipo do rastreador e a contagem de frames
        cv2.imshow("Tracking", frame)

        # o script é terminado se a tecla esc for pressionada
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break

## Resultado - KCF

In [2]:
from IPython.display import Video
# Vídeo Base: Monologo de Grasiela Camargo - https://www.youtube.com/watch?v=AbrTiNZ7wd4
Video("result1.mkv")

## Resultado MIL

In [1]:
from IPython.display import Video
# Vídeo Base: Monologo de Grasiela Camargo - https://www.youtube.com/watch?v=AbrTiNZ7wd4
Video("result2.mkv")

# Comentários

Apesar do tracker MIL aparentar oscilar menor a caixa de seleção durante o vídeo a sua performance em quesito de quadros por segundo foi imensamente inferior ao KCF, com uma taxa média de 25 e 55 quadros por segundos respectivamente.